In [1]:
import pandas as pd
import numpy as np
import statistics
def genXDayAverage(df, days, tag):
    toReturn = np.array([0] * df.shape[0])
    toReturn = pd.Series(toReturn, dtype = 'float64')
    for x in range(days):
        shiftedSeries = df[tag].shift(x)
        toReturn = toReturn.add(shiftedSeries)
    return toReturn.div(days)

def stdDevLastXDays(df, days,tag):
    toReturn = []
    pastXDayList = [np.nan] * days
    for _,item in df[tag].items():
        pastXDayList.append(item)
        pastXDayList.pop(0)
        try:
            toReturn.append(statistics.stdev(pastXDayList))
        except:
            toReturn.append(np.nan)
    toReturn = pd.Series(toReturn)
    return toReturn

In [2]:
import json
import requests
import pandas as pd
import numpy as np

apiKey = ''
with open('config/fredKeys.json', 'r') as f:
    data = json.load(f)
    apiKey = data['APIKEY']
fileRoot = 'fredData/'
urlRoot = 'https://api.stlouisfed.org/fred/series/observations?'
seriesOfInterest = ['DTWEXAFEGS','DPRIME', 'TOTCI', 'UNRATE', 'CONSUMER','BUSLOANS','CCLACBW027SBOG']
dateRange = np.arange('2010-01-01', '2020-05-26', dtype='datetime64[D]')
dateRange = {'Date': dateRange}
dateDF = pd.DataFrame(data = dateRange)
seriesDFList= []
for series in seriesOfInterest:
    url = urlRoot + 'series_id='+ series + '&api_key='+ apiKey + '&file_type=json'
    resp = requests.get(url)
    obs = resp.json()['observations']
    dateList = [x['date'] for x in obs]
    valList = [x['value'] for x in obs]
    valueLabel = series + 'Value'
    newDF = {'Date': dateList, valueLabel: valList}
    newDF = pd.DataFrame(data = newDF)
    newDF['Date'] = pd.to_datetime(newDF['Date'])
    
    newDF.replace(['.'], np.nan, inplace = True)
    newDF.fillna(method = 'ffill', inplace = True)
    newDF[valueLabel] = newDF[valueLabel].astype('float64')
    
    newDF[series + 'lastChangeP'] = (newDF[valueLabel] - newDF[valueLabel].shift(1)).div(newDF[valueLabel].shift(1))
    fiveTickAverage = genXDayAverage(newDF, 5, valueLabel)
    tenTickAverage = genXDayAverage(newDF, 10, valueLabel)
    newDF[series + 'fiveVsTenTickAverage'] = (fiveTickAverage - tenTickAverage).div(tenTickAverage)
    
    ticks = 10
    Std = stdDevLastXDays(newDF, ticks,valueLabel)
    Average = genXDayAverage(newDF, ticks, valueLabel)
    upperBand = ((Std).multiply(2) + Average)
    lowerBand = ((Std).multiply(-2) + Average)
    bPercent = []
    for close, upper, lower in zip (newDF[valueLabel], upperBand, lowerBand):
        if upper - lower == 0:
            bPercent.append(0)
        else:
            bPercent.append((close - lower)/(upper - lower))
    newDF[series + 'bPercent'] = bPercent
    newDF = dateDF.merge(newDF, right_on = 'Date', left_on = 'Date', how = 'left')
    seriesDFList.append(newDF)
masterDF = seriesDFList[0].copy()
counter = 0
for df in seriesDFList:
    if counter != 0:
        masterDF= masterDF.merge(df, right_on = 'Date', left_on = 'Date')
    else:
        counter += 1
        


In [3]:
masterDF.fillna(method = 'ffill', inplace = True)
df = pd.read_csv('masterDF.csv', parse_dates = True)
df['Date'] = pd.to_datetime(df['Date'])
df = df.merge(masterDF, right_on = 'Date', left_on = 'Date')
df.to_csv('combinedDF.csv', index=False) 